<a href="https://colab.research.google.com/github/EthanBrewerCity/NLP_CW_210008199/blob/main/NLP_CW_210008199.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##NLP Project: Ethan Brewer <br>
Dataset: Dair-ai/emotion <br>
Project Type: Sentiment Analysis <br>


# NLP Models: DistilBERT and LSTM
Add Setup Features for Project <br>
This Includes device setup, installs, imports and loading the original dataset

In [7]:
# Installs necessary
!pip install transformers
!pip install datasets

In [8]:
# Add imports
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Further imports via pytorch
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence
from torch.optim import AdamW as Adam # Import AdamW optimiser as previous instance was "deprecated"

# Imports from prior pip installs
from datasets import load_dataset
from transformers import DistilBertModel, DistilBertTokenizer, AdamW # AdamW on this version is stated as "Deprecated" so other version from torch.optim is used

# Sklearn imports
from sklearn.model_selection import learning_curve
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Load Dataset: dair-ai/emotion from Huggingface
dataset = load_dataset('emotion', trust_remote_code=True)

# Ensure the device being used is either the cuda or cpu (if cuda isn't available)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# CODE FOR BERT BEGINS BELOW

## Preprocessing the Data
The Tokeniser and Model are taken and the data is preprocessed via BERT (Currently Distilled)

In [9]:
# Load Tokeniser and Model for dataset, In this case, BERT (Distilled vers. may change later)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


## Create the Neural Network and Define the Model Architecture

In [10]:
# Defining Model Architecture
class SentiClass(nn.Module):
  def __init__(self,model):
    super(SentiClass, self).__init__()
    self.model = model
    self.linear = nn.Linear(768,6)
  def forward(self,input_ids, attention_mask):
    outputs = self.model(input_ids = input_ids, attention_mask = attention_mask)
    last_hidden_state = outputs[0]
    logits = self.linear(last_hidden_state[:,0,:])
    return logits

# Creating intstance of the model
model = SentiClass(model)

## Define the Train, Eval and Eval for confusion matrix

In [11]:
# Tokenize the data
def tokenize(batch):
    return tokenizer(batch['text'], padding=True, truncation=True)

train_dataset = dataset['train'].map(tokenize, batched=True)
test_dataset = dataset['test'].map(tokenize, batched=True)

# Set the data format
train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

# Set up the optimizer and loss function
optimizer = Adam(model.parameters(), lr=5e-5,weight_decay=0.01)
# Old Loss function, not usable as it is binary
#loss_fn = nn.BCEWithLogitsLoss()
#New loss function
loss_fn = nn.CrossEntropyLoss()

# Define the training loop and prepare data for loss graph
def train(model, train_loader, optimizer, loss_fn, device,num_epochs):
    model.train()
    train_losses=[]
    for epoch in range(num_epochs):
      # Sets epoch loss to 0 to calculate the value for each epoch
      epoch_loss= 0.0
      for batch in train_loader:
          input_ids = batch['input_ids'].to(device)
          attention_mask = batch['attention_mask'].to(device)
          labels = batch['label'].to(device)
          optimizer.zero_grad()
          outputs = model(input_ids, attention_mask)
          loss = loss_fn(outputs, labels.long())
          # Debug print to check the loss, commented to keep displayed results a bit cleaner
          #print(loss)
          loss.backward()
          optimizer.step()
          # Adds loss to variable
          epoch_loss+=loss.item()
      # Calculates average loss in each epoch
      epoch_loss /= len(train_loader)
      # Append epoch loss to array
      train_losses.append(epoch_loss)
      print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss}")
    return train_losses

# Define the evaluation loop
def evaluate(model, test_loader, device):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)

            outputs = model(input_ids, attention_mask)
            predictions = torch.argmax(outputs,dim=1)

            total += labels.size(0)
            correct += (predictions == labels).sum().item()

    return correct / total

def eval_conf_matrix(model, test_loader, device):
  model.eval()
  all_preds = []
  all_labels = []
  with torch.no_grad():
    for batch in test_loader:
      input_ids = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      labels = batch['label'].to(device)
      #Debug print for labels to ensure they match the intended result
      #print("Labels:", labels)

      outputs = model(input_ids, attention_mask)
      # Debug prints to check outputs and their shape, prior issue with shape comparison, now fixed
      #print("Outputs:", outputs)
      #print("Outputs shape:", outputs.shape)
      predictions = torch.argmax(outputs, dim=1).cpu().numpy()

      all_preds.extend(predictions)
      all_labels.extend(labels.cpu().numpy())

  ## Check expected range of class indices
  #print("Min Label:", np.min(all_labels))
  #print("Max Label:", np.max(all_labels))

  ## Debug prints to show labels
  #print("True Labels:", all_labels)
  #print("Predictions:", all_preds)

  return all_labels, all_preds

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


##Training the model, adding padding and producing results for BERT

In [12]:
# Train the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Pad the sequences
def collate_fn(batch):
  input_ids = [item['input_ids'] for item in batch]
  attention_mask = [item['attention_mask'] for item in batch]
  labels = [item['label'] for item in batch]

  input_ids = pad_sequence(input_ids, batch_first=True, padding_value=tokenizer.pad_token_id)
  attention_mask = pad_sequence(attention_mask,batch_first=True,padding_value=0)

  return {
      'input_ids': input_ids,
      'attention_mask': attention_mask,
      'label': torch.tensor(labels)
  }

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle = True, collate_fn = collate_fn)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle = False, collate_fn = collate_fn)

# Set number of epochs - Used later for data representation displayed
epoch_val = 5

# Train is placed into variable names history for use in data representation later
history = train(model, train_loader, optimizer, loss_fn, device,epoch_val)
print(history)
# Accuracy calculated and used for confusion matrix later
accuracy = evaluate(model, test_loader, device)
print(f'Test Accuracy: {accuracy:.3f}')

# Loss graph for training
plt.figure(1)
plt.plot(range(1, epoch_val + 1), history , label="Training Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.title("Training Loss vs Epochs")
plt.legend()
plt.show()


# Confusion matrix
plt.figure(2)
true_labels, predictions = eval_conf_matrix(model, test_loader, device)

# Print confusion matrix - Debug test
#print("Confusion Matrix:", conf_mat)

conf_mat = confusion_matrix(true_labels,predictions)
plt.figure(figsize = (6,6))
sns.heatmap(conf_mat, annot=True, fmt="d", cmap="Blues",
            xticklabels=dataset['train'].features['label'].names,
            yticklabels=dataset['train'].features['label'].names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

# Classsification report
# Print Classification Report
print("Classification Report:")
print(classification_report(true_labels, predictions))
## Old code for model, epochs now performed within the train loop and accuracy performed after the train is done
#for epoch in range(5):
#    history = train(model, train_loader, optimizer, loss_fn, device,epoch_val)
#    accuracy = evaluate(model, test_loader, device)
#    print(f'Epoch {epoch+1} - Test Accuracy: {accuracy:.3f}')


Epoch 1/5, Loss: 0.37002445592544975
Epoch 2/5, Loss: 0.12637029287125914
Epoch 3/5, Loss: 0.10099903567088768
Epoch 4/5, Loss: 0.08753712428011931
Epoch 5/5, Loss: 0.06704715734662023
[0.37002445592544975, 0.12637029287125914, 0.10099903567088768, 0.08753712428011931, 0.06704715734662023]
Test Accuracy: 0.925


<Figure size 640x480 with 0 Axes>

Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.96      0.96       581
           1       0.98      0.91      0.94       695
           2       0.74      0.96      0.84       159
           3       0.93      0.94      0.94       275
           4       0.94      0.83      0.88       224
           5       0.64      0.95      0.77        66

    accuracy                           0.93      2000
   macro avg       0.87      0.93      0.89      2000
weighted avg       0.94      0.93      0.93      2000



# CODE FOR BERT ENDS AND Naive BEGINS

## Model 2:
Step 1: Imports and installs

In [13]:
# Installs necessary
!pip install transformers
!pip install datasets

In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from datasets import load_dataset
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt

Loading dataset

In [15]:
dataset = load_dataset('emotion',trust_remote_code = True)
train_data = dataset['train']
test_data = dataset['test']
# Loading validation for cross fold
val_data = dataset['validation']

Step 4: Vectorisation of data

In [16]:
# Uses countVectoriser() to tokenise and vectorise data
# Hyperparameters for count vectoriser
# Best values currently for accuracy are 8,0.85,1,3
min_df_val = 8
max_df_val = 0.85
ngram_val_min = 1
ngram_val_max = 3
# Count vectoriser with hyperparams
vectoriser = CountVectorizer(min_df=min_df_val, max_df=max_df_val, ngram_range=(ngram_val_min, ngram_val_max))
# Setting values into data for checking
X_train = vectoriser.fit_transform(train_data["text"])
X_test = vectoriser.transform(test_data["text"])
X_val = vectoriser.fit_transform(val_data["text"])
y_train = train_data["label"]
y_test = test_data["label"]
y_val = val_data["label"]

 Step 3: Creation of model and hyperparameters involved

In [17]:
# Make the model a Naive Bayes multinomial for the 6 potential labels
# Setting Hyperparameter in order to  tweak results
# Best value right now for alpha with other hyperparams is 0.45
alpha_val = 0.45
model = MultinomialNB(alpha=alpha_val)

Step 4: Fitting model

In [18]:
model.fit(X_train, y_train)

MultinomialNB(alpha=0.45)

Step 4.5: Validation of dataset

In [19]:
# Cross-validation used, hyperparam for cv
# Not the best result for val but produces best result for accuracy is 10 fold
cross_fold_val = 10
acc_scores = cross_val_score(model, X_val, y_val, cv=cross_fold_val)
print("Cross-validation scores:", acc_scores)
print("Mean accuracy:", acc_scores.mean())

Cross-validation scores: [0.525 0.505 0.55  0.49  0.565 0.52  0.55  0.555 0.555 0.505]
Mean accuracy: 0.5319999999999999


Step 5: Evaluation of model and resulting accuracy

In [20]:
y_predict = model.predict(X_test)
accuracy = accuracy_score(y_test, y_predict)
print("Accuracy:", accuracy)
# Best Accuracy right now is 0.797
cm = confusion_matrix(y_test, y_predict)

Accuracy: 0.797


Step 6: Displaying results

In [21]:
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=dataset["train"].features["label"].names, yticklabels=dataset["train"].features["label"].names)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.show()

# Print Classification Report
print("Classification Report:")
print(classification_report(y_test, y_predict))

Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.84      0.85       581
           1       0.85      0.86      0.86       695
           2       0.57      0.64      0.60       159
           3       0.79      0.75      0.77       275
           4       0.71      0.71      0.71       224
           5       0.55      0.55      0.55        66

    accuracy                           0.80      2000
   macro avg       0.72      0.73      0.72      2000
weighted avg       0.80      0.80      0.80      2000

